In [2]:
raw_lines = [{"_airbyte_raw_id":"019c32ec-a81f-7e7f-8294-bc0f88573fcb","_airbyte_extracted_at":1770380895000,"_airbyte_meta":{"sync_id":5,"changes":[]},"_airbyte_generation_id":1,"_airbyte_data":{"Wind":"WSW","Temperature":"56.8 °F","Solar":"0 w/m²","Humidity":"87 %","Precip. Accum.":"0.00 in","Time":"00:04:00","Precip. Rate.":"0.00 in","UV":0,"Pressure":"29.48 in","Dew Point":"53.1 °F","Gust":"10.4 mph","Speed":"8.2 mph"}},
{"_airbyte_raw_id":"019c32ec-a826-75fc-ae3c-6739d7d90eb8","_airbyte_extracted_at":1770380895000,"_airbyte_meta":{"sync_id":5,"changes":[]},"_airbyte_generation_id":1,"_airbyte_data":{"Wind":"WSW","Temperature":"56.8 °F","Solar":"0 w/m²","Humidity":"87 %","Precip. Accum.":"0.00 in","Time":"00:09:00","Precip. Rate.":"0.00 in","UV":0,"Pressure":"29.47 in","Dew Point":"52.9 °F","Gust":"9.8 mph","Speed":"7.9 mph"}}]

for idx, line in enumerate(raw_lines, start=1):
            airbyte_data = line.get("_airbyte_data")
            print(airbyte_data)

{'Wind': 'WSW', 'Temperature': '56.8\xa0°F', 'Solar': '0\xa0w/m²', 'Humidity': '87\xa0%', 'Precip. Accum.': '0.00\xa0in', 'Time': '00:04:00', 'Precip. Rate.': '0.00\xa0in', 'UV': 0, 'Pressure': '29.48\xa0in', 'Dew Point': '53.1\xa0°F', 'Gust': '10.4\xa0mph', 'Speed': '8.2\xa0mph'}
{'Wind': 'WSW', 'Temperature': '56.8\xa0°F', 'Solar': '0\xa0w/m²', 'Humidity': '87\xa0%', 'Precip. Accum.': '0.00\xa0in', 'Time': '00:09:00', 'Precip. Rate.': '0.00\xa0in', 'UV': 0, 'Pressure': '29.47\xa0in', 'Dew Point': '52.9\xa0°F', 'Gust': '9.8\xa0mph', 'Speed': '7.9\xa0mph'}


In [7]:

from datetime import datetime
from typing import Dict, Any, Optional
from loguru import logger
import re

from typing import Any, Optional
from datetime import datetime
import re

def parse_timestamp(timestamp: Any, file_date: Optional[str] = None) -> Optional[str]:
    if timestamp is None:
        return None

    ts_str = str(timestamp).strip().replace("\xa0", " ")

    # 👉 NOUVEAU : format US avec AM/PM
    am_pm_formats = [
        "%m/%d/%y %I:%M %p",
        "%m/%d/%y %I:%M:%S %p",
    ]
    for fmt in am_pm_formats:
        try:
            dt = datetime.strptime(ts_str, fmt)
            return dt.isoformat()
        except ValueError:
            pass

    # Cas heure seule, combiner avec la date du fichier
    if re.fullmatch(r"\d{2}:\d{2}(:\d{2})?", ts_str) and file_date:
        ts_str = f"{file_date} {ts_str}"

    ts_str = ts_str.replace(" ", "T")

    # Formats standards
    formats = [
        "%Y-%m-%dT%H:%M:%S",
        "%Y-%m-%dT%H:%M",
        "%Y-%m-%d",
    ]

    for fmt in formats:
        try:
            dt = datetime.strptime(ts_str, fmt)
            return dt.isoformat()
        except ValueError:
            continue

    # fallback ISO
    try:
        dt = datetime.fromisoformat(ts_str)
        return dt.isoformat()
    except Exception:
        return None


In [8]:
print(parse_timestamp('01/10/24 12:09 AM'))

2024-01-10T00:09:00
